# 콜백 (Callback)

Model을 학습시킬 때 부가적으로 옵션을 넣어서 수행할 수 있도록 도와주는 객체

모델 학습시 사용하는 fit()함수에 callbacks라는 파라미터로 지정할 수 있음


In [23]:
import tensorflow as tf
from tensorflow import keras

In [24]:
# keras 내장 datasets에서 mnist 불러오기
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [25]:
# data 정규화 0~1사이로
x_train = x_train / x_train.max()
x_test = x_test / 255.

In [26]:
# model 생성

model = keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(256,activation='relu'),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(32,activation='relu'),
    keras.layers.Dense(10,activation='softmax')
])

In [27]:
# model compile
model.compile(optimizer='adam',
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 체크포인트 (Model Checkpoint)

epoch별로 model의 가중치를 저장하는 역할

체크포인트 하이퍼파라미터
  * filepath : 체크포인트 저장하는 경로를 지정
  * save_weights_only : 가중치만 저장할 지 여부
  (True, False)
  * save_best_only : monitor 기준 가장 높은 epoch만 저장할지 or
 
  epoch마다 저장할지 여부
  (True, False)
  * monitor : 저장 시 기준이 되는 지표를 설정
  * verbase : epoch마다 저장 여부를 알려주는 로그메시지 출력여부(1, 0)

In [28]:
#체크포인트 설정
checkPoint = keras.callbacks.ModelCheckpoint(
    filepath = 'checkPoint.ckpt', # ckpt : 모델의 가중치 체크포인트 저장 파일
    # 텐서플로 모델 구조 자체는 제외한 가중치'만' 담고 있는 파일
    # 이 파일을 통해서 재학습이 가능, 불필요한 정보도 있어서 파일의 크기가 크고 무거움
    save_weight_only=True, # 가중치만 저장하겠다.
    # False : 모델의 층(layer), weights 모두 저장
    save_best_only=True,
    # True : monitor되고 있는 값 기준으로 가장 뛰어난 epoch 모델이 저장
    # False : epoch마다 모델이 filepath{epoch} 형태로 저장
    monitor='val_loss', # model을 저장할 때 기준이 되는 값을 지정
    # 테스트 셋을 기준으로 loss가 가장 적을 때 저장하려면 'val_loss'
    # 학습 셋을 기준으로 loss가 가장 적을 때 저장하려면 'loss'
    verbose=1 # 1이면 모델 저장시 문구가 뜸, 0이면 문구 없이 저장만 뜸
)

In [29]:
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          epochs=10,
          callbacks=[checkPoint])

Epoch 1/10
1867/1875 [============================>.] - ETA: 0s - loss: 0.2362 - accuracy: 0.9296
Epoch 1: val_loss improved from inf to 0.12338, saving model to checkPoint.ckpt
1875/1875 [==============================] - 12s 6ms/step - loss: 0.2358 - accuracy: 0.9297 - val_loss: 0.1234 - val_accuracy: 0.9609
Epoch 2/10
1874/1875 [============================>.] - ETA: 0s - loss: 0.0973 - accuracy: 0.9701
Epoch 2: val_loss improved from 0.12338 to 0.08356, saving model to checkPoint.ckpt
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0973 - accuracy: 0.9701 - val_loss: 0.0836 - val_accuracy: 0.9747
Epoch 3/10
1871/1875 [============================>.] - ETA: 0s - loss: 0.0681 - accuracy: 0.9779
Epoch 3: val_loss improved from 0.08356 to 0.07507, saving model to checkPoint.ckpt
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0680 - accuracy: 0.9779 - val_loss: 0.0751 - val_accuracy: 0.9764
Epoch 4/10
1868/1875 [============================>.] - 

In [30]:
# 체크포인트 적용 전 모델
loss, acc = model.evaluate(x_test,y_test)
print('loss : ', loss, 'accuracy : ', acc)

# 체크포인트 적용 전 모델
model.load_weights('checkPoint.ckpt')
#.load_weights를 사용해서 불러오지 않으면 저장된 가중치는 로드되지 않음
loss, acc = model.evaluate(x_test,y_test)
print('loss : ', loss, 'accuracy : ', acc)

# 모델 체크포인트에 저장된 가중치를 불러왔을 때에는
# 가장 검증손실이 낮았던 epoch 실행 후의 결과가 나옴
# 검증손실이 가장 낮았던 모델 가중치를 사용할 수 있다.

313/313 [==============================] - 1s 3ms/step - loss: 0.0950 - accuracy: 0.9779
loss :  0.09500793367624283 accuracy :  0.9779000282287598
313/313 [==============================] - 1s 2ms/step - loss: 0.0734 - accuracy: 0.9796
loss :  0.0733979195356369 accuracy :  0.9796000123023987
